# 1) OBTAINING AMERICAN STOCKS LIST FROM YAHOO FINANCE


In [ ]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from bs4 import BeautifulSoup
import time

# Initialize an empty DataFrame outside the function
df = pd.DataFrame()


def get_data(link):
    global df  # Add this line to refer to the global variable df inside the function
    chrome_driver = ChromeDriverManager().install()
    driver = Chrome(service=Service(chrome_driver))
    driver.get(link)
    time.sleep(1)
    accept_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Accetta tutto')]")
    accept_button.click()
    time.sleep(1)
    data = []
    screener_results_div = driver.find_element(By.ID, "screener-results")
    screener_results_html = screener_results_div.get_attribute("innerHTML")
    soup = BeautifulSoup(screener_results_html, 'html.parser')
    rows = soup.find_all('tr')
    for row in rows:
        cols = row.find_all(['th', 'td'])
        cols = [col.text.strip() for col in cols]
        data.append(cols)
    data = [row for row in data if row]
    # Concatenate the data with the global DataFrame df
    df = pd.concat([df, pd.DataFrame(data[1:], columns=data[0])])


for i in range(0, 9900+100, 100):
    link = 'https://finance.yahoo.com/screener/unsaved/ee6758fa-5ef1-4c88-a096-4b6f14942ad3?count=100&offset=' + str(i)
    get_data(link)

print(df)
# You should quit the driver after you have collected all data.
driver.quit()

df.to_csv('us_stocks.csv', index=False)

# 2) IMPORTING OPTIONS DATASET


In [3]:
import pandas as pd

stocks = pd.read_csv('us_stocks.csv')
stocks

tickers_symbol = list(stocks['Symbol'])

In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime


data = pd.DataFrame()  # Initialize an empty DataFrame


for ticker_symbol in tickers_symbol:
    try:
        ticker = yf.Ticker(ticker_symbol)
        option_chain = ticker.option_chain()
        try:
            ticker_calls = option_chain.calls  # Extract calls data
            ticker_calls['ticker'] = ticker_symbol  # Add a column for ticker symbol
            ticker_calls['type'] = "Call" # Add a column for ticker symbol
            data = pd.concat([data, ticker_calls])  # Concatenate calls data to the DataFrame
        except:
            pass
        
        try:
            ticker_puts = option_chain.puts  # Extract puts data
            ticker_puts['ticker'] = ticker_symbol  # Add a column for ticker symbol
            ticker_puts['type'] = "Put" # Add a column for ticker symbol
            data = pd.concat([data, ticker_puts])  # Concatenate puts data to the DataFrame
        except:
            pass
    except:
        pass
data

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,ticker,type
0,MSFT240328C00210000,2024-03-08 14:58:25+00:00,210.0,199.70,209.70,210.45,0.000000,0.000000,3.0,0.0,3.773438,1.0,REGULAR,USD,MSFT,Call
1,MSFT240328C00220000,2024-03-27 15:47:29+00:00,220.0,200.32,199.60,200.10,-9.569992,-4.559527,2.0,2.0,3.593751,1.0,REGULAR,USD,MSFT,Call
2,MSFT240328C00230000,2024-03-27 16:15:35+00:00,230.0,189.90,189.55,190.35,1.399994,0.742702,3.0,3.0,2.898440,1.0,REGULAR,USD,MSFT,Call
3,MSFT240328C00235000,2024-02-13 16:24:52+00:00,235.0,174.91,188.55,191.70,0.000000,0.000000,NaN,6.0,5.750003,1.0,REGULAR,USD,MSFT,Call
4,MSFT240328C00250000,2024-03-27 15:51:15+00:00,250.0,170.23,169.50,170.40,-9.120010,-5.085035,1.0,6.0,2.523441,1.0,REGULAR,USD,MSFT,Call
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,UXIN240517C00002000,2024-01-10 14:55:12+00:00,2.0,0.05,0.05,0.75,0.000000,0.000000,1.0,1382.0,0.828127,True,REGULAR,USD,UXIN,Call
2,UXIN240517C00003000,2023-11-16 14:30:00+00:00,3.0,0.15,0.00,0.10,0.000000,0.000000,6.0,12.0,0.718753,False,REGULAR,USD,UXIN,Call
3,UXIN240517C00004000,2023-09-19 14:48:38+00:00,4.0,0.16,0.00,0.90,0.000000,0.000000,NaN,1.0,2.535160,False,REGULAR,USD,UXIN,Call
0,UXIN240517P00001000,2023-12-06 15:54:06+00:00,1.0,0.25,0.35,0.55,0.000000,0.000000,NaN,1.0,4.578129,False,REGULAR,USD,UXIN,Put


In [8]:
data.to_csv('data.csv', index=False)

# 3) CLEANING OPTION DATA

In [5]:
import pandas as pd

data = pd.read_csv('data_to_clean.csv',low_memory=False)
data

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,ticker,type
0,MSFT240328C00210000,2024-03-08 14:58:25+00:00,210.0,199.70,209.70,210.45,0.000000,0.000000,3.0,0.0,3.773438,1.0,REGULAR,USD,MSFT,Call
1,MSFT240328C00220000,2024-03-27 15:47:29+00:00,220.0,200.32,199.60,200.10,-9.569992,-4.559527,2.0,2.0,3.593751,1.0,REGULAR,USD,MSFT,Call
2,MSFT240328C00230000,2024-03-27 16:15:35+00:00,230.0,189.90,189.55,190.35,1.399994,0.742702,3.0,3.0,2.898440,1.0,REGULAR,USD,MSFT,Call
3,MSFT240328C00235000,2024-02-13 16:24:52+00:00,235.0,174.91,188.55,191.70,0.000000,0.000000,NaN,6.0,5.750003,1.0,REGULAR,USD,MSFT,Call
4,MSFT240328C00250000,2024-03-27 15:51:15+00:00,250.0,170.23,169.50,170.40,-9.120010,-5.085035,1.0,6.0,2.523441,1.0,REGULAR,USD,MSFT,Call
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73739,UXIN240517C00002000,2024-01-10 14:55:12+00:00,2.0,0.05,0.05,0.75,0.000000,0.000000,1.0,1382.0,0.828127,True,REGULAR,USD,UXIN,Call
73740,UXIN240517C00003000,2023-11-16 14:30:00+00:00,3.0,0.15,0.00,0.10,0.000000,0.000000,6.0,12.0,0.718753,False,REGULAR,USD,UXIN,Call
73741,UXIN240517C00004000,2023-09-19 14:48:38+00:00,4.0,0.16,0.00,0.90,0.000000,0.000000,NaN,1.0,2.535160,False,REGULAR,USD,UXIN,Call
73742,UXIN240517P00001000,2023-12-06 15:54:06+00:00,1.0,0.25,0.35,0.55,0.000000,0.000000,NaN,1.0,4.578129,False,REGULAR,USD,UXIN,Put


#### Extract expiration date from contractSymbol

In [6]:
from datetime import datetime


def extract_expiration(row):
    index = len(row['ticker'])
    expiration_str = row['contractSymbol']
    try:
        expiration_date = datetime.strptime(expiration_str[index: index + 6], "%y%m%d").date()
    except:
        expiration_date = None
    return expiration_date

# Apply function to each row
data['expirationDate'] = data.apply(extract_expiration, axis=1)
data

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,ticker,type,expirationDate
0,MSFT240328C00210000,2024-03-08 14:58:25+00:00,210.0,199.70,209.70,210.45,0.000000,0.000000,3.0,0.0,3.773438,1.0,REGULAR,USD,MSFT,Call,2024-03-28
1,MSFT240328C00220000,2024-03-27 15:47:29+00:00,220.0,200.32,199.60,200.10,-9.569992,-4.559527,2.0,2.0,3.593751,1.0,REGULAR,USD,MSFT,Call,2024-03-28
2,MSFT240328C00230000,2024-03-27 16:15:35+00:00,230.0,189.90,189.55,190.35,1.399994,0.742702,3.0,3.0,2.898440,1.0,REGULAR,USD,MSFT,Call,2024-03-28
3,MSFT240328C00235000,2024-02-13 16:24:52+00:00,235.0,174.91,188.55,191.70,0.000000,0.000000,NaN,6.0,5.750003,1.0,REGULAR,USD,MSFT,Call,2024-03-28
4,MSFT240328C00250000,2024-03-27 15:51:15+00:00,250.0,170.23,169.50,170.40,-9.120010,-5.085035,1.0,6.0,2.523441,1.0,REGULAR,USD,MSFT,Call,2024-03-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73739,UXIN240517C00002000,2024-01-10 14:55:12+00:00,2.0,0.05,0.05,0.75,0.000000,0.000000,1.0,1382.0,0.828127,True,REGULAR,USD,UXIN,Call,2024-05-17
73740,UXIN240517C00003000,2023-11-16 14:30:00+00:00,3.0,0.15,0.00,0.10,0.000000,0.000000,6.0,12.0,0.718753,False,REGULAR,USD,UXIN,Call,2024-05-17
73741,UXIN240517C00004000,2023-09-19 14:48:38+00:00,4.0,0.16,0.00,0.90,0.000000,0.000000,NaN,1.0,2.535160,False,REGULAR,USD,UXIN,Call,2024-05-17
73742,UXIN240517P00001000,2023-12-06 15:54:06+00:00,1.0,0.25,0.35,0.55,0.000000,0.000000,NaN,1.0,4.578129,False,REGULAR,USD,UXIN,Put,2024-05-17


#### Extract date from lastTradeDate

In [7]:
def extract_data(lastTradeDate):
    data_datetime = datetime.strptime(str(lastTradeDate), "%Y-%m-%d %H:%M:%S%z")
    return data_datetime.date()

# Apply function to each row
data['lastTradeDate'] = data['lastTradeDate'].apply(extract_data)
data

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,ticker,type,expirationDate
0,MSFT240328C00210000,2024-03-08,210.0,199.70,209.70,210.45,0.000000,0.000000,3.0,0.0,3.773438,1.0,REGULAR,USD,MSFT,Call,2024-03-28
1,MSFT240328C00220000,2024-03-27,220.0,200.32,199.60,200.10,-9.569992,-4.559527,2.0,2.0,3.593751,1.0,REGULAR,USD,MSFT,Call,2024-03-28
2,MSFT240328C00230000,2024-03-27,230.0,189.90,189.55,190.35,1.399994,0.742702,3.0,3.0,2.898440,1.0,REGULAR,USD,MSFT,Call,2024-03-28
3,MSFT240328C00235000,2024-02-13,235.0,174.91,188.55,191.70,0.000000,0.000000,NaN,6.0,5.750003,1.0,REGULAR,USD,MSFT,Call,2024-03-28
4,MSFT240328C00250000,2024-03-27,250.0,170.23,169.50,170.40,-9.120010,-5.085035,1.0,6.0,2.523441,1.0,REGULAR,USD,MSFT,Call,2024-03-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73739,UXIN240517C00002000,2024-01-10,2.0,0.05,0.05,0.75,0.000000,0.000000,1.0,1382.0,0.828127,True,REGULAR,USD,UXIN,Call,2024-05-17
73740,UXIN240517C00003000,2023-11-16,3.0,0.15,0.00,0.10,0.000000,0.000000,6.0,12.0,0.718753,False,REGULAR,USD,UXIN,Call,2024-05-17
73741,UXIN240517C00004000,2023-09-19,4.0,0.16,0.00,0.90,0.000000,0.000000,NaN,1.0,2.535160,False,REGULAR,USD,UXIN,Call,2024-05-17
73742,UXIN240517P00001000,2023-12-06,1.0,0.25,0.35,0.55,0.000000,0.000000,NaN,1.0,4.578129,False,REGULAR,USD,UXIN,Put,2024-05-17


#### Deleting useless variables

In [8]:
data.drop('contractSymbol', axis=1, inplace=True)
data.drop('currency', axis=1, inplace=True)

#### Getting stock prices


In [ ]:
from datetime import datetime, timedelta
import yfinance as yf

def stock_price(row):
    end = datetime.strptime(str(row['lastTradeDate']), "%Y-%m-%d") + timedelta(days=10)
    start = end - timedelta(days=20)

    ticker = row['ticker']
    stock = yf.Ticker(ticker)
    try:
        historical_data = stock.history(start=start, end=end)
        closest_date = historical_data.index.min()
        closest_price = round(historical_data.loc[closest_date]['Close'], 2)
    except:
        closest_price = False
    return closest_price


data['stock'] = data.apply(stock_price, axis=1)
data = data[data['stock'] != 'False']
data['stock'] = data['stock'].astype('float')

# 4) INCLUDING STOCK INFO

#### Importing data

In [ ]:
ticker_list = data['ticker'].unique()

info = {}
for ticker in ticker_list:
    company = yf.Ticker(ticker)
    company_info = company.info
    
    # Delete unwanted keys
    keys_to_remove = ['phone', 'website', 'longBusinessSummary', 'companyOfficers', 
                      'currency', 'symbol', 'underlyingSymbol', 'shortName', 
                      'longName', 'financialCurrency']
    for key in keys_to_remove:
        company_info.pop(key, None)
    
    info[ticker] = company_info  # Use the ticker symbol as the key

In [ ]:
for index, row in data.iterrows():
    ticker = row['ticker']
    company_info = info.get(ticker)  # Retrieve company information using ticker symbol
    if company_info:
        for key, value in company_info.items():
            data.loc[index, key] = value  # Assign company information to DataFrame

#### Deleting variables with missing values > 50%

In [3]:
import pandas as pd

data = pd.read_csv('full_data.csv',low_memory=False)

In [ ]:
missing_percentage = (data.isnull().sum() / len(data)) * 100

# Filtra solo le colonne con valori mancanti
columns_to_drop = missing_percentage[missing_percentage > 50]
data = data.drop(columns=columns_to_drop)

#### Replacing missing values with 0 for dividend varables (non-dividend paying-stocks)

In [6]:
to_fill_0 = [
    'dividendRate',
    'dividendYield',
    'exDividendDate',
    'payoutRatio',
    'fiveYearAvgDividendYield',
    'trailingAnnualDividendRate',
    'trailingAnnualDividendYield',
    'impliedSharesOutstanding',
    'lastDividendValue',
    'lastDividendDate', 
]

data[to_fill_0] = data[to_fill_0].fillna(0)


#### Replace missing values with median couse they are not normal distributed

In [ ]:
from scipy import stats
    
import matplotlib.pyplot as plt
import numpy as np

columns = [
    'auditRisk',
    'boardRisk', 
    'compensationRisk',
    'shareHolderRightsRisk',
    'overallRisk'
]
for column in columns:
    data_column = data[column].astype('float') # I tuoi dati qui


    stats.probplot(data_column, dist="norm", plot=plt)
    plt.title("Q-Q plot " + str(column))
    plt.show()

In [8]:
to_fill_median =[
    'auditRisk',
    'boardRisk', 
    'compensationRisk',
    'shareHolderRightsRisk',
    'overallRisk', 
]

column_median = data[to_fill_median].median()
data[to_fill_median] = data[to_fill_median].fillna(column_median)


# 5) FORMATTING AND ENCODING VARIABLES

#### Obtaining tau (resitual time to expiration) instead of the expiration date

In [10]:
import pandas as pd
import numpy as np

# Assuming 'data' is a DataFrame containing 'lastTradeDate' and 'expirationDate' columns

def tau(row):
    try:
        start_date = pd.to_datetime(row['lastTradeDate']).to_numpy().astype('datetime64[D]')
        end_date = pd.to_datetime(row['expirationDate']).to_numpy().astype('datetime64[D]')
    except ValueError:
        return np.nan
    
    residual_days = np.busday_count(start_date, end_date)
    tau_value = residual_days / 252
    return tau_value

def tau2(row):
    try:
        start_date = pd.to_datetime(row['lastTradeDate']).to_numpy().astype('datetime64[D]')
        end_date = pd.to_datetime(row['expirationDate']).to_numpy().astype('datetime64[D]')
    except ValueError:
        return np.nan
    
    residual_days = (end_date - start_date).astype('timedelta64[D]').astype(int)
    tau_value = residual_days / 365
    return tau_value

# Assuming 'data' is your DataFrame
data['tau'] = data.apply(tau, axis=1)
data['tau2'] = data.apply(tau2, axis=1)

data.dropna(subset=['expirationDate'], inplace=True)


In [ ]:
missing_percentage = (data.isnull().sum() / len(data)) * 100

# Filtra solo le colonne con valori mancanti
missing_percentage = missing_percentage[missing_percentage > 5]
missing_percentage

In [21]:
data.to_csv('full_data_2.csv')

# 6)  COMPUTING VOLATILITY

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

volatility_dic = {}

for ticker in ticker_list:

    stock_data = yf.download(ticker, start='2023-03-27', end='2024-03-27')

    if 'Close' in stock_data.columns:
        stock_data['Daily Return'] = stock_data['Close'].pct_change()
        average_daily_return = stock_data['Daily Return'].mean()
        daily_volatility = stock_data['Daily Return'].std()
        annual_volatility = daily_volatility * np.sqrt(252)

        volatility_dic[ticker] = annual_volatility
    else:
        print("Error")



In [50]:
from datetime import datetime, timedelta
import yfinance as yf

def volatility(row):

    ticker = row['ticker']
    return volatility_dic[ticker]


data['sigma'] = data.apply(volatility, axis=1)

#### Controlling that no stocks has 0 or negative volatility

In [72]:
to_correct = data.ticker[data.sigma.isnull()].unique()

In [76]:
stock_data['Daily Return'] = stock_data['Close'].pct_change()
average_daily_return = stock_data['Daily Return'].mean()
daily_volatility = stock_data['Daily Return'].std()
annual_volatility = daily_volatility * np.sqrt(252)

volatility_dic[ticker] = annual_volatility

{'FLT': nan,
 'RDDT': 2.3707587136471258,
 'MDC': nan,
 'CBAY': nan,
 'TCN': nan,
 'PGTI': nan,
 'CPE': nan,
 'DWAC': nan,
 'INBX': nan,
 'KAMN': nan,
 'NGMS': nan,
 'EGLE': nan,
 'AMNB': nan,
 'CSTR': nan,
 'DSKE': nan,
 'CCLP': nan,
 'MIXT': nan,
 'CBD': nan,
 'CVCY': nan,
 'GRPH': nan,
 'FSR': nan,
 'NGM': nan,
 'KNTE': nan,
 'SCTL': nan,
 'FIXX': nan,
 'CASA': nan,
 'SOLO': nan,
 'AAU': nan,
 'ONTX': nan}

#### Removing stocks with null volatility

In [85]:
data = data.dropna(subset=['sigma'])

# 7) Final result

In [86]:
data

,lastTradeDate,strike,price,bid,ask,change,percentChange,volume,openInterest,sigma,...,returnOnAssets,returnOnEquity,freeCashflow,operatingCashflow,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins,tau,tau2
0,2024-03-08,210.0,199.70,417.49,418.06,0.000000,0.000000,19275689.0,0.0,0.223118,...,0.15190,0.39174,5.868100e+10,1.026470e+11,0.176,0.69815,0.52037,0.43586,0.055556,0.054795
1,2024-03-27,220.0,200.32,417.49,418.06,-9.569992,-4.559527,19275689.0,2.0,0.223118,...,0.15190,0.39174,5.868100e+10,1.026470e+11,0.176,0.69815,0.52037,0.43586,0.003968,0.002740
2,2024-03-27,230.0,189.90,417.49,418.06,1.399994,0.742702,19275689.0,3.0,0.223118,...,0.15190,0.39174,5.868100e+10,1.026470e+11,0.176,0.69815,0.52037,0.43586,0.003968,0.002740
3,2024-02-13,235.0,174.91,417.49,418.06,0.000000,0.000000,19275689.0,6.0,0.223118,...,0.15190,0.39174,5.868100e+10,1.026470e+11,0.176,0.69815,0.52037,0.43586,0.126984,0.120548
4,2024-03-27,250.0,170.23,417.49,418.06,-9.120010,-5.085035,19275689.0,6.0,0.223118,...,0.15190,0.39174,5.868100e+10,1.026470e+11,0.176,0.69815,0.52037,0.43586,0.003968,0.002740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73561,2024-01-10,2.0,0.05,1.71,2.34,0.000000,0.000000,124821.0,1382.0,1.064334,...,-0.10972,0.10401,2.431477e+08,5.283660e+08,-0.425,0.03447,-0.18544,-0.18645,0.365079,0.350685
73562,2023-11-16,3.0,0.15,1.71,2.34,0.000000,0.000000,124821.0,12.0,1.064334,...,-0.10972,0.10401,2.431477e+08,5.283660e+08,-0.425,0.03447,-0.18544,-0.18645,0.519841,0.501370
73563,2023-09-19,4.0,0.16,1.71,2.34,0.000000,0.000000,124821.0,1.0,1.064334,...,-0.10972,0.10401,2.431477e+08,5.283660e+08,-0.425,0.03447,-0.18544,-0.18645,0.686508,0.660274
73564,2023-12-06,1.0,0.25,1.71,2.34,0.000000,0.000000,124821.0,1.0,1.064334,...,-0.10972,0.10401,2.431477e+08,5.283660e+08,-0.425,0.03447,-0.18544,-0.18645,0.464286,0.446575
